In [70]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner



In [ ]:
data = pd.read_csv('./culture.csv')

In [67]:
def delete_stop_words(words):
    clean_words = [] 
    preprocessor = cleaner.Preproccess()
    stop_words  = preprocessor.get_stop_words()
    for i in words:
        if i  not in stop_words:
            clean_words.append(i)
    return clean_words


In [258]:
sentence = nltk.sent_tokenize(unicode(data['mText'][1],'utf-8'))
sentSeri = pd.Series(sentence)
words = nltk.word_tokenize(sentence[0])

In [259]:
pd.Series(sentence[1].split(' '))

0         بنا
1          بر
2         این
3      گزارش،
4         این
5       تصمیم
6          پس
7          از
8      درگیری
9        لفظی
10        آرش
11    ظلی‌پور
12         با
13      مسعود
14     فراستی
15         در
16     برنامه
17        روز
18       جمعه
19      گرفته
20        شد.
dtype: object

In [263]:
sent = re.sub(r'\s.{2,10}$','',sentence[1])

In [264]:
words = nltk.word_tokenize(sent)
words = delete_stop_words(words)

In [265]:
pd.Series(words)

0         بنا
1      گزارش،
2       تصمیم
3      درگیری
4        لفظی
5         آرش
6     ظلی‌پور
7       مسعود
8      فراستی
9      برنامه
10       جمعه
dtype: object